In [ ]:
import subprocess, os, sys, time, importlib

os.chdir('/workspaces/scrapermeleegg')
sys.path.insert(0, '/workspaces/scrapermeleegg')

# Kill any existing process on port 8080
subprocess.run('kill $(lsof -t -i:8080) 2>/dev/null', shell=True)
time.sleep(1)

# Regenerate the site
import generate_site
importlib.reload(generate_site)
import manage_tournaments as mt
importlib.reload(mt)

db = mt.TournamentDB()
dist_path = '/workspaces/scrapermeleegg/dist'
if db.tournaments:
    generate_site.generate_multi_tournament_site(db.data['tournaments'], dist_path)
    size = os.path.getsize(os.path.join(dist_path, 'index.html'))
    print(f'Site generated: {size} bytes')

with open(os.path.join(dist_path, 'index.html'), 'r') as f:
    html = f.read()
for term in ['home-panel', 'add-box', 'Scrapear', 'btn-do-add']:
    print(f'  {term}: {html.count(term)}')

server = subprocess.Popen(
    [sys.executable, 'server.py', '--port', '8080'],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
    cwd='/workspaces/scrapermeleegg'
)
time.sleep(2)
print(f'Server PID: {server.pid}')

r = subprocess.run('curl -sI http://localhost:8080/ | head -10', shell=True, capture_output=True, text=True)
print(r.stdout)
print('Done: http://localhost:8080')

In [ ]:
import subprocess, os
os.chdir('/workspaces/scrapermeleegg')

# Stage all changed files
files = ['generate_site.py', 'server.py', 'launch.py', 'launch.ipynb', 
         'restart.ipynb', 'run_server.ipynb', 'diagnose.py', 'melee_research.ipynb']
for f in files:
    subprocess.run(['git', 'add', f], capture_output=True)

status = subprocess.run(['git', 'status', '--short'], capture_output=True, text=True)
print('Staged files:')
print(status.stdout)

# Commit
commit = subprocess.run(
    ['git', 'commit', '-m', 
     'feat: home panel with tournament cards + API server\n\n'
     '- Replace dropdown selector with full home panel (tournament cards, add box, progress bar)\n'
     '- Add server.py with REST API (add/remove tournaments from browser)\n'
     '- Add active tournament bar for navigation between home and analysis\n'
     '- Add launch.py, restart.ipynb, run_server.ipynb helper scripts\n'
     '- Keyboard shortcuts: 0=home, 1=metagame, 2=matchups, 3=detail'],
    capture_output=True, text=True
)
print('\nCommit:')
print(commit.stdout or commit.stderr)

# Push
push = subprocess.run(['git', 'push', 'origin', 'main'], capture_output=True, text=True, timeout=30)
print('\nPush:')
print(push.stdout or push.stderr)

In [2]:
import subprocess, os
os.chdir('/workspaces/scrapermeleegg')

# Stage files
files = ['generate_site.py', 'server.py', 'launch.py', 'launch.ipynb', 
         'restart.ipynb', 'run_server.ipynb', 'diagnose.py', 'melee_research.ipynb']
for f in files:
    subprocess.run(['git', 'add', f], capture_output=True, text=True)

# Verify staged
status = subprocess.run(['git', 'status', '--short'], capture_output=True, text=True, timeout=10)
print('Staged:')
print(status.stdout)

Staged:
A  diagnose.py
M  generate_site.py
A  launch.ipynb
A  launch.py
M  melee_research.ipynb
A  restart.ipynb
A  run_server.ipynb
A  server.py



In [3]:
import subprocess
# Commit
commit = subprocess.run(
    ['git', 'commit', '-m', 
     'feat: home panel with tournament cards + API server\n\n'
     '- Replace dropdown selector with full home panel (tournament cards, add box, progress bar)\n'
     '- Add server.py with REST API (add/remove tournaments from browser)\n'
     '- Add active tournament bar for navigation between home and analysis\n'
     '- Add launch.py, restart.ipynb, run_server.ipynb helper scripts\n'
     '- Keyboard shortcuts: 0=home, 1=metagame, 2=matchups, 3=detail'],
    capture_output=True, text=True, timeout=15
)
print('Commit:')
print(commit.stdout or commit.stderr)

# Push
push = subprocess.run(['git', 'push', 'origin', 'main'], capture_output=True, text=True, timeout=60)
print('\nPush:')
print(push.stdout or push.stderr)

Commit:
[main 201ee7d] feat: home panel with tournament cards + API server
 8 files changed, 1157 insertions(+), 45 deletions(-)
 create mode 100644 diagnose.py
 create mode 100644 launch.ipynb
 create mode 100644 launch.py
 create mode 100644 restart.ipynb
 create mode 100644 run_server.ipynb
 create mode 100644 server.py


Push:
To https://github.com/torinescudo/scrapermeleegg
   d9ced0a..201ee7d  main -> main



In [4]:
import subprocess

# 1. Check if server is running
r = subprocess.run(['curl', '-s', 'http://localhost:8080/api/status'], capture_output=True, text=True, timeout=5)
print('Server status:', r.stdout)

# 2. Check current tournaments
r2 = subprocess.run(['curl', '-s', 'http://localhost:8080/api/tournaments'], capture_output=True, text=True, timeout=5)
print('\nCurrent tournaments:', r2.stdout[:500])

Server status: 

Current tournaments: 


In [5]:
import subprocess, os, sys, time, importlib

os.chdir('/workspaces/scrapermeleegg')
sys.path.insert(0, '/workspaces/scrapermeleegg')

# Kill any leftover
subprocess.run('kill $(lsof -t -i:8080) 2>/dev/null', shell=True)
time.sleep(1)

# Regenerate site
import generate_site
importlib.reload(generate_site)
import manage_tournaments as mt
importlib.reload(mt)

db = mt.TournamentDB()
dist_path = '/workspaces/scrapermeleegg/dist'
if db.tournaments:
    generate_site.generate_multi_tournament_site(db.data['tournaments'], dist_path)
    print(f'Site generated: {os.path.getsize(os.path.join(dist_path, "index.html"))} bytes')
else:
    print('No tournaments in DB yet')

# Launch server in background
server = subprocess.Popen(
    [sys.executable, 'server.py', '--port', '8080'],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
    cwd='/workspaces/scrapermeleegg'
)
time.sleep(2)

# Verify
r = subprocess.run(['curl', '-s', 'http://localhost:8080/api/status'], capture_output=True, text=True, timeout=5)
print(f'Server PID: {server.pid}')
print(f'Status: {r.stdout}')

/workspaces/scrapermeleegg/generate_site.py:1235: SyntaxWarning: invalid escape sequence '\.'
  </html>"""


[*] Procesando: Regional Championship - SCG CON Milwaukee - Season 4 - Round 2 - Saturday - 9:00 am
    Arquetipos con partidas: 104

[✓] Sitio generado en: /workspaces/scrapermeleegg/dist/
    index.html  (743,491 bytes)
    Torneos embebidos: 1
      • Regional Championship - SCG CON Milwaukee - Season 4 - Round 2 - Saturday - 9:00 am (5046 matches)
Site generated: 746015 bytes
Server PID: 12943
Status: {"ok": true, "status": "running"}


In [6]:
import subprocess, json, time

# Test: add tournament via API (same one that's already loaded)
print('=== TEST: POST /api/tournaments/add-sync ===')
print('URL: https://melee.gg/Tournament/View/339227')
print()

start = time.time()
r = subprocess.run([
    'curl', '-s', '-X', 'POST',
    '-H', 'Content-Type: application/json',
    '-d', '{"url": "https://melee.gg/Tournament/View/339227"}',
    'http://localhost:8080/api/tournaments/add-sync'
], capture_output=True, text=True, timeout=180)
elapsed = time.time() - start

print(f'Time: {elapsed:.1f}s')
print(f'Raw response ({len(r.stdout)} bytes):')

try:
    data = json.loads(r.stdout)
    print(json.dumps(data, indent=2, ensure_ascii=False))
    
    if data.get('ok'):
        print(f'\n✅ Resultado: {data.get("action")}')
        print(f'   Nombre: {data.get("name")}')
        print(f'   Matches: {data.get("total_matches")}')
        print(f'   Mensaje: {data.get("message")}')
    else:
        print(f'\n❌ Error: {data.get("error")}')
except:
    print(r.stdout[:2000])
    
if r.stderr:
    print(f'\nstderr: {r.stderr[:500]}')

=== TEST: POST /api/tournaments/add-sync ===
URL: https://melee.gg/Tournament/View/339227

Time: 0.0s
Raw response (304 bytes):
{
  "ok": true,
  "action": "already_exists",
  "tournament_id": "339227",
  "name": "Regional Championship - SCG CON Milwaukee - Season 4 - Round 2 - Saturday - 9:00 am",
  "total_matches": 5046,
  "message": "Torneo ya existente: Regional Championship - SCG CON Milwaukee - Season 4 - Round 2 - Saturday - 9:00 am"
}

✅ Resultado: already_exists
   Nombre: Regional Championship - SCG CON Milwaukee - Season 4 - Round 2 - Saturday - 9:00 am
   Matches: 5046
   Mensaje: Torneo ya existente: Regional Championship - SCG CON Milwaukee - Season 4 - Round 2 - Saturday - 9:00 am


In [7]:
import subprocess, json

# Test GET /api/tournaments
print('=== GET /api/tournaments ===')
r = subprocess.run(['curl', '-s', 'http://localhost:8080/api/tournaments'], capture_output=True, text=True, timeout=5)
data = json.loads(r.stdout)
print(f'Count: {data["count"]}')
for t in data['tournaments']:
    print(f'  [{t["id"]}] {t["name"]} — {t["total_matches"]} matches')

# Test invalid URL
print('\n=== TEST: Invalid URL ===')
r2 = subprocess.run([
    'curl', '-s', '-X', 'POST',
    '-H', 'Content-Type: application/json',
    '-d', '{"url": "https://google.com"}',
    'http://localhost:8080/api/tournaments/add-sync'
], capture_output=True, text=True, timeout=10)
data2 = json.loads(r2.stdout)
print(f'ok={data2["ok"]}, error={data2.get("error", "none")}')

# Test empty URL
print('\n=== TEST: Empty URL ===')
r3 = subprocess.run([
    'curl', '-s', '-X', 'POST',
    '-H', 'Content-Type: application/json',
    '-d', '{"url": ""}',
    'http://localhost:8080/api/tournaments/add-sync'
], capture_output=True, text=True, timeout=10)
data3 = json.loads(r3.stdout)
print(f'ok={data3["ok"]}, error={data3.get("error", "none")}')

print('\n✅ Todos los endpoints funcionan correctamente')

=== GET /api/tournaments ===
Count: 1
  [339227] Regional Championship - SCG CON Milwaukee - Season 4 - Round 2 - Saturday - 9:00 am — 5046 matches

=== TEST: Invalid URL ===
ok=False, error=No se pudo extraer un ID de torneo de: 'https://google.com'
Formatos válidos:
  - https://melee.gg/Tournament/View/339227
  - 339227

=== TEST: Empty URL ===
ok=False, error=Falta el campo 'url'

✅ Todos los endpoints funcionan correctamente


In [8]:
import subprocess
r = subprocess.run(['git', 'ls-files', 'tournaments_db.json'], capture_output=True, text=True)
print(f'tournaments_db.json tracked: {"YES" if r.stdout.strip() else "NO"}')

r2 = subprocess.run(['wc', '-c', 'tournaments_db.json'], capture_output=True, text=True)
print(f'Size: {r2.stdout.strip()}')

tournaments_db.json tracked: YES
Size: 2161350 tournaments_db.json


In [9]:
import subprocess, os

os.chdir('/workspaces/scrapermeleegg')

# Stage
subprocess.run(['git', 'add', 'generate_site.py', 'netlify.toml'], capture_output=True)
r = subprocess.run(['git', 'status', '--short'], capture_output=True, text=True)
print('Staged:', r.stdout)

# Commit
commit = subprocess.run(
    ['git', 'commit', '-m', 
     'fix: configure Netlify build + static mode detection\n\n'
     '- Add build command to netlify.toml (python generate_site.py)\n'
     '- Detect static mode (no API): hide scraping UI and delete buttons\n'
     '- CSS class .static-mode hides server-only features automatically'],
    capture_output=True, text=True, timeout=15
)
print('\nCommit:', commit.stdout or commit.stderr)

# Push
push = subprocess.run(['git', 'push', 'origin', 'main'], capture_output=True, text=True, timeout=60)
print('\nPush:', push.stdout or push.stderr)

Staged: M  generate_site.py
M  netlify.toml
 M restart.ipynb


Commit: [main bb3c7f9] fix: configure Netlify build + static mode detection
 2 files changed, 15 insertions(+), 3 deletions(-)


Push: To https://github.com/torinescudo/scrapermeleegg
   201ee7d..bb3c7f9  main -> main



In [10]:
import subprocess

# Check what Netlify returns
r = subprocess.run(['curl', '-sL', '-w', '\n--- HTTP %{http_code} ---\nContent-Type: %{content_type}\nSize: %{size_download} bytes\n', 
                     'https://scrappermeleegg.netlify.app/'], 
                    capture_output=True, text=True, timeout=30)
print('Response body (first 2000 chars):')
print(r.stdout[:2000])
print('\n...')
print(r.stdout[-500:])

if r.stderr:
    print('\nstderr:', r.stderr[:500])

Response body (first 2000 chars):
<!DOCTYPE html>
<html lang="es">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>MTG Meta Analyzer</title>
<link rel="icon" href="data:image/svg+xml,<svg xmlns='http://www.w3.org/2000/svg' viewBox='0 0 100 100'><text y='.9em' font-size='90'>🎴</text></svg>">
<style>
/* ═══════════════ RESET & BASE ═══════════════ */
*,*::before,*::after{box-sizing:border-box;margin:0;padding:0}
:root{
  --bg-0:#0b0d14;--bg-1:#111320;--bg-2:#181b2c;--bg-3:#1f2340;
  --border:#2a2f4a;--border-focus:#4f46e5;
  --text-0:#f0f0f5;--text-1:#c5c7d6;--text-2:#8b8fa8;--text-3:#5c6080;
  --accent:#6366f1;--accent-hover:#818cf8;
  --green:#34d399;--yellow:#fbbf24;--red:#f87171;
  --good:rgba(52,211,153,.12);--ok:rgba(251,191,36,.08);--bad:rgba(248,113,113,.12);
  --radius:8px;--shadow:0 4px 24px rgba(0,0,0,.4);
  --transition:all .2s ease;
}
html{scroll-behavior:smooth}
body{font-family:'Inter',-apple-system,BlinkMacSystem

In [11]:
import subprocess, json

# Download full HTML and analyze it
r = subprocess.run(['curl', '-sL', 'https://scrappermeleegg.netlify.app/'], 
                    capture_output=True, text=True, timeout=30)
html = r.stdout
print(f'HTML size: {len(html)} bytes')

# Check critical elements exist
checks = {
    'DOCTYPE': '<!DOCTYPE html>' in html,
    'home-panel': 'home-panel' in html,
    'meta-panel': 'meta-panel' in html,
    'ALL_TOURNAMENTS': 'ALL_TOURNAMENTS' in html,
    'static-mode CSS': 'static-mode' in html,
    'loadTournament': 'loadTournament' in html,
    'renderHomeTournaments': 'renderHomeTournaments' in html,
    'switchPanel': 'switchPanel' in html,
}
for k, v in checks.items():
    print(f'  {"✅" if v else "❌"} {k}')

# Extract the ALL_TOURNAMENTS data to see if it has content
import re
m = re.search(r'const ALL_TOURNAMENTS = (\{.*?\});\s*const PIE_COLORS', html, re.DOTALL)
if m:
    try:
        tournaments = json.loads(m.group(1))
        tids = list(tournaments.keys())
        print(f'\nTournaments embedded: {len(tids)}')
        for tid in tids:
            t = tournaments[tid]
            print(f'  [{tid}] {t.get("tournament", {}).get("name", "?")}')
            print(f'    archetypes: {len(t.get("archetypes", []))}')
            print(f'    matrix_decks: {len(t.get("matrix_decks", []))}')
            print(f'    total_pilots: {t.get("total_pilots", "?")}')
    except Exception as e:
        print(f'JSON parse error: {e}')
        print(f'JSON preview: {m.group(1)[:500]}')
else:
    print('\n❌ ALL_TOURNAMENTS not found in HTML!')

# Check for JS syntax errors - look for unescaped characters
script_start = html.find('<script>')
script_end = html.rfind('</script>')
if script_start > 0 and script_end > script_start:
    js_code = html[script_start+8:script_end]
    print(f'\nJS code length: {len(js_code)} chars')
    
    # Check for common issues
    # Unclosed braces
    open_braces = js_code.count('{')
    close_braces = js_code.count('}')
    print(f'  Braces balance: {{ {open_braces} vs }} {close_braces} {"✅" if open_braces == close_braces else "❌ MISMATCH!"}')

HTML size: 743868 bytes
  ✅ DOCTYPE
  ✅ home-panel
  ✅ meta-panel
  ✅ ALL_TOURNAMENTS
  ✅ static-mode CSS
  ✅ loadTournament
  ✅ renderHomeTournaments
  ✅ switchPanel

Tournaments embedded: 1
  [339227] Regional Championship - SCG CON Milwaukee - Season 4 - Round 2 - Saturday - 9:00 am
    archetypes: 104
    matrix_decks: 104
    total_pilots: 1193

JS code length: 719639 chars
  Braces balance: { 11231 vs } 11232 ❌ MISMATCH!


In [12]:
import subprocess, re

# Download and extract JS
r = subprocess.run(['curl', '-sL', 'https://scrappermeleegg.netlify.app/'], 
                    capture_output=True, text=True, timeout=30)
html = r.stdout

script_start = html.find('<script>')
script_end = html.rfind('</script>')
js = html[script_start+8:script_end]

# Track brace balance line by line to find the mismatch
lines = js.split('\n')
balance = 0
issues = []
for i, line in enumerate(lines):
    open_b = line.count('{')
    close_b = line.count('}')
    balance += open_b - close_b
    if balance < 0:
        issues.append((i+1, balance, line.strip()[:120]))
        
if issues:
    print(f'Found {len(issues)} lines where balance goes negative:')
    for linenum, bal, text in issues[:10]:
        print(f'  Line {linenum}: balance={bal}')
        print(f'    {text}')
else:
    print('No negative balance found, checking end...')

print(f'\nFinal balance: {balance}')
print(f'Expected: 0')

# Look at last 30 lines of JS
print('\n--- Last 30 lines of JS ---')
for i, line in enumerate(lines[-30:], len(lines)-29):
    print(f'{i:5d}: {line[:120]}')

Found 70 lines where balance goes negative:
  Line 209: balance=-1
    }
  Line 210: balance=-1
    
  Line 211: balance=-1
    // Sort on header click
  Line 220: balance=-1
    });
  Line 221: balance=-1
    
  Line 222: balance=-1
    document.getElementById('f-min').addEventListener('input', renderMetaTable);
  Line 223: balance=-1
    document.getElementById('f-search').addEventListener('input', renderMetaTable);
  Line 224: balance=-1
    
  Line 225: balance=-1
    /* Render home panel tournament list */
  Line 291: balance=-1
    }

Final balance: -1
Expected: 0

--- Last 30 lines of JS ---
  636:         showStatus(`🎉 ¡Añadido! <strong>${data.name}</strong> — ${data.total_matches} partidas, ${data.total_players || 
  637:       }
  638:       setTimeout(() => window.location.reload(), 1500);
  639:       return;
  640:     } else {
  641:       document.querySelectorAll('.progress-step.active').forEach(s => s.className = 'progress-step error');
  642:       setProgress(0, 'Err

In [13]:
import subprocess
r = subprocess.run(['curl', '-sL', 'https://scrappermeleegg.netlify.app/'], 
                    capture_output=True, text=True, timeout=30)
html = r.stdout
script_start = html.find('<script>')
script_end = html.rfind('</script>')
js = html[script_start+8:script_end]
lines = js.split('\n')

# Show lines 195-215 where the issue is
print('--- Lines 195-220 of JS (around the brace mismatch) ---')
for i in range(194, min(220, len(lines))):
    open_b = lines[i].count('{')
    close_b = lines[i].count('}')
    marker = ' <<< CLOSE' if close_b > open_b else ''
    print(f'{i+1:4d}: {lines[i][:130]}{marker}')

--- Lines 195-220 of JS (around the brace mismatch) ---
 195:       document.getElementById('dd-select').value = name;
 196:       document.querySelector('.nav-btn[data-panel="detail-panel"]').click();
 197:     }); <<< CLOSE
 198:   }); <<< CLOSE
 199: 
 200:   renderPie(filtered);
 201:   document.getElementById('pie-total').textContent = filtered.length;
 202: 
 203:   // Update header sort indicators
 204:     th.classList.remove('sorted-asc', 'sorted-desc');
 205:     if (th.dataset.key === metaSortKey) {
 206:       th.classList.add(metaSortDir > 0 ? 'sorted-asc' : 'sorted-desc');
 207:     } <<< CLOSE
 208:   }); <<< CLOSE
 209: } <<< CLOSE
 210: 
 211: // Sort on header click
 212: document.querySelectorAll('#meta-tbl thead th[data-key]').forEach(th => {
 213:   th.addEventListener('click', () => {
 214:     const key = th.dataset.key;
 215:     if (key === 'idx') return;
 216:     if (metaSortKey === key) metaSortDir *= -1;
 217:     else { metaSortKey = key; metaSortDir = key

In [14]:
import importlib, os, sys, json
sys.path.insert(0, '/workspaces/scrapermeleegg')
os.chdir('/workspaces/scrapermeleegg')

import generate_site, manage_tournaments as mt
importlib.reload(generate_site)
importlib.reload(mt)

db = mt.TournamentDB()
generate_site.generate_multi_tournament_site(db.data['tournaments'], 'dist')

# Verify brace balance
with open('dist/index.html', 'r') as f:
    html = f.read()

script_start = html.find('<script>')
script_end = html.rfind('</script>')
js = html[script_start+8:script_end]

open_b = js.count('{')
close_b = js.count('}')
print(f'\nBrace balance: {{ {open_b} vs }} {close_b} {"✅ MATCHED!" if open_b == close_b else "❌ MISMATCH!"}')

# Quick JS validation: check no negative balance
lines = js.split('\n')
balance = 0
first_negative = None
for i, line in enumerate(lines):
    balance += line.count('{') - line.count('}')
    if balance < 0 and first_negative is None:
        first_negative = i + 1
        
print(f'Final balance: {balance}')
if first_negative:
    print(f'First negative at line {first_negative}')
else:
    print('No negative balance found ✅')

[*] Procesando: Regional Championship - SCG CON Milwaukee - Season 4 - Round 2 - Saturday - 9:00 am
    Arquetipos con partidas: 104

[✓] Sitio generado en: dist/
    index.html  (743,944 bytes)
    Torneos embebidos: 1
      • Regional Championship - SCG CON Milwaukee - Season 4 - Round 2 - Saturday - 9:00 am (5046 matches)

Brace balance: { 11232 vs } 11232 ✅ MATCHED!
Final balance: 0
No negative balance found ✅


/workspaces/scrapermeleegg/generate_site.py:1247: SyntaxWarning: invalid escape sequence '\.'
  for tid, tdata in tournaments.items():


In [15]:
import subprocess
subprocess.run(['git', 'add', 'generate_site.py', 'netlify.toml'], capture_output=True)

commit = subprocess.run(
    ['git', 'commit', '-m', 
     'fix: JS brace mismatch breaking Netlify deploy\n\n'
     '- Add missing forEach loop for header sort indicators\n'
     '- Fix regex escape warning in f-string\n'
     '- Brace balance now correct (11232/11232)'],
    capture_output=True, text=True, timeout=15
)
print('Commit:', commit.stdout or commit.stderr)

push = subprocess.run(['git', 'push', 'origin', 'main'], capture_output=True, text=True, timeout=60)
print('Push:', push.stdout or push.stderr)

Commit: [main a104103] fix: JS brace mismatch breaking Netlify deploy
 1 file changed, 2 insertions(+), 1 deletion(-)

Push: To https://github.com/torinescudo/scrapermeleegg
   bb3c7f9..a104103  main -> main



In [16]:
import time
print('Esperando 60s a que Netlify redeploy...')
time.sleep(60)

import subprocess
r = subprocess.run(['curl', '-sL', 'https://scrappermeleegg.netlify.app/'], 
                    capture_output=True, text=True, timeout=30)
html = r.stdout
print(f'HTML size: {len(html)} bytes')

# Check brace balance
script_start = html.find('<script>')
script_end = html.rfind('</script>')
if script_start > 0:
    js = html[script_start+8:script_end]
    ob = js.count('{')
    cb = js.count('}')
    print(f'Brace balance: {{ {ob} vs }} {cb} {"✅" if ob == cb else "❌"}')
    
    # Check if the fix is deployed (look for the forEach we added)
    if 'meta-tbl thead th[data-key]').forEach' in js[:15000]:
        print('✅ Fix is deployed (forEach present in sort indicators)')
    
    # Quick functional check: look for the renderHomeTournaments call
    if 'renderHomeTournaments()' in js:
        print('✅ renderHomeTournaments call present')

# Check key elements
for term in ['home-panel', 'ALL_TOURNAMENTS', 'static-mode']:
    print(f'  {term}: {"✅" if term in html else "❌"}')

SyntaxError: unmatched ')' (826944668.py, line 21)

In [17]:
import time
print('Esperando 60s a que Netlify redeploy...')
time.sleep(60)

import subprocess
r = subprocess.run(['curl', '-sL', 'https://scrappermeleegg.netlify.app/'], 
                    capture_output=True, text=True, timeout=30)
html = r.stdout
print(f'HTML size: {len(html)} bytes')

script_start = html.find('<script>')
script_end = html.rfind('</script>')
if script_start > 0:
    js = html[script_start+8:script_end]
    ob = js.count('{')
    cb = js.count('}')
    print(f'Brace balance: open={ob} close={cb} {"MATCHED" if ob == cb else "MISMATCH!"}')
    
    if 'renderHomeTournaments()' in js:
        print('renderHomeTournaments: present')

for term in ['home-panel', 'ALL_TOURNAMENTS', 'static-mode']:
    found = 'YES' if term in html else 'NO'
    print(f'  {term}: {found}')

Esperando 60s a que Netlify redeploy...
HTML size: 743945 bytes
Brace balance: open=11232 close=11232 MATCHED
renderHomeTournaments: present
  home-panel: YES
  ALL_TOURNAMENTS: YES
  static-mode: YES


In [18]:
import subprocess, json, re

# Download the deployed site
r = subprocess.run(['curl', '-sL', 'https://scrappermeleegg.netlify.app/'], 
                    capture_output=True, text=True, timeout=30)
html = r.stdout

# Extract the JS
script_start = html.find('<script>')
script_end = html.rfind('</script>')
js = html[script_start+8:script_end]

# Check ALL_TOURNAMENTS content
m = re.search(r'const ALL_TOURNAMENTS = (\{.*?\});\s*const PIE_COLORS', html, re.DOTALL)
if m:
    tournaments = json.loads(m.group(1))
    print(f'Tournaments: {len(tournaments)}')
    for tid, t in tournaments.items():
        print(f'  [{tid}] archetypes={len(t["archetypes"])}, matrix={len(t["matrix_decks"])}')
        # Show first 5 archetypes
        for a in t["archetypes"][:5]:
            print(f'    {a["name"]}: {a["share"]}% share, {a["winrate"]}% WR, {a["total"]} matches')

# Check if renderHomeTournaments function exists and is called
print(f'\nrenderHomeTournaments defined: {"function renderHomeTournaments" in js}')
print(f'renderHomeTournaments called: {"renderHomeTournaments()" in js}')

# Check renderMetaTable 
print(f'renderMetaTable defined: {"function renderMetaTable" in js}')

# Look for the t-grid rendering
idx = js.find('function renderHomeTournaments')
if idx >= 0:
    func_code = js[idx:idx+2000]
    print(f'\n--- renderHomeTournaments function ---')
    print(func_code[:1500])
    
# Check if static-mode class is being set
print(f'\nstatic-mode class set: {"classList.add" in js and "static-mode" in js}')

# Check the CSS for static-mode
css_start = html.find('<style>')
css_end = html.find('</style>')
css = html[css_start:css_end]
print(f'CSS static-mode .add-box display:none: {"static-mode .add-box" in css}')
print(f'CSS static-mode .t-card-del display:none: {"static-mode .t-card-del" in css}')

Tournaments: 1
  [339227] archetypes=104, matrix=104
    Izzet Lessons: 10.73% share, 53.06% WR, 1159 matches
    Mono-Green Landfall: 8.72% share, 54.68% WR, 1026 matches
    Dimir Excruciator: 8.47% share, 47.57% WR, 824 matches
    Dimir Midrange: 6.62% share, 50.73% WR, 684 matches
    Simic Rhythm: 6.45% share, 49.53% WR, 644 matches

renderHomeTournaments defined: True
renderHomeTournaments called: True
renderMetaTable defined: True

--- renderHomeTournaments function ---
function renderHomeTournaments() {
  const grid = document.getElementById('t-grid');
  const tids = Object.keys(ALL_TOURNAMENTS);
  
  if (tids.length === 0) {
    grid.innerHTML = '<div class="t-empty">No hay torneos cargados todavía.<br>Usa el formulario de arriba para añadir uno.</div>';
    document.getElementById('t-grid-title').textContent = '';
    return;
  }

  document.getElementById('t-grid-title').textContent = tids.length + ' torneo' + (tids.length !== 1 ? 's' : '') + ' disponible' + (tids.length !=

In [19]:
import subprocess
# Use node to check JS syntax
r = subprocess.run(['curl', '-sL', 'https://scrappermeleegg.netlify.app/'], 
                    capture_output=True, text=True, timeout=30)
html = r.stdout

script_start = html.find('<script>')
script_end = html.rfind('</script>')
js = html[script_start+8:script_end]

# Write JS to temp file and check syntax with node
with open('/tmp/check.js', 'w') as f:
    f.write(js)

r2 = subprocess.run(['node', '--check', '/tmp/check.js'], capture_output=True, text=True, timeout=10)
print('Node --check exit code:', r2.returncode)
if r2.stderr:
    print('Errors:', r2.stderr[:2000])
else:
    print('No syntax errors!')

Node --check exit code: 0
No syntax errors!


In [20]:
import subprocess, re

r = subprocess.run(['curl', '-sL', 'https://scrappermeleegg.netlify.app/'], 
                    capture_output=True, text=True, timeout=30)
html = r.stdout

# Check if the old version (without fix) is still cached
# Look for the forEach in sort indicators area
if 'Update header sort indicators' in html:
    idx = html.index('Update header sort indicators')
    snippet = html[idx:idx+400]
    print('=== Sort indicators area (check forEach is there) ===')
    print(snippet)

# Check meta-tbl structure
print('\n=== meta-tbl in HTML ===')
meta_tbl_matches = re.findall(r'id="meta-tbl"[^>]*>', html)
print(f'meta-tbl found: {len(meta_tbl_matches)}')
for m in meta_tbl_matches:
    print(f'  {m}')

# Check the metagame table HTML
idx2 = html.find('id="meta-tbl"')
if idx2 > 0:
    print(html[idx2:idx2+1000])

# Check if there's a renderMetaTable call
print('\n=== renderMetaTable calls ===')
for m in re.finditer(r'renderMetaTable\(\)', html):
    # Show context
    start = max(0, m.start()-80)
    end = min(len(html), m.end()+20)
    print(f'  ...{html[start:end]}...')

# Check if the old version might still be served (look for the missing forEach)
print('\n=== Check if fix is deployed ===')
idx3 = html.find('Update header sort indicators')
area = html[idx3:idx3+300] if idx3 >= 0 else 'NOT FOUND'
has_foreach = 'forEach' in area
print(f'forEach in sort indicators: {has_foreach}')

=== Sort indicators area (check forEach is there) ===
Update header sort indicators
  document.querySelectorAll('#meta-tbl thead th[data-key]').forEach(th => {
    th.classList.remove('sorted-asc', 'sorted-desc');
    if (th.dataset.key === metaSortKey) {
      th.classList.add(metaSortDir > 0 ? 'sorted-asc' : 'sorted-desc');
    }
  });
}

// Sort on header click
document.querySelectorAll('#meta-tbl thead th[data-key]').forEach(th => {
  th.addEvent

=== meta-tbl in HTML ===
meta-tbl found: 1
  id="meta-tbl">
id="meta-tbl">
              <thead>
                <tr>
                  <th data-key="idx" style="width:40px">#</th>
                  <th data-key="name">Archetype</th>
                  <th data-key="share">Meta&nbsp;%</th>
                  <th data-key="count" class="num">Pilots</th>
                  <th data-key="wins" class="num">W</th>
                  <th data-key="losses" class="num">L</th>
                  <th data-key="draws" class="num">D</th>
                 

In [21]:
import subprocess

r = subprocess.run(['curl', '-sL', 'https://scrappermeleegg.netlify.app/'], 
                    capture_output=True, text=True, timeout=30)
html = r.stdout
script_start = html.find('<script>')
script_end = html.rfind('</script>')
js = html[script_start+8:script_end]

# Check order of key definitions vs usage
lines = js.split('\n')

# Find where esc function is defined
for i, line in enumerate(lines):
    if 'function esc(' in line or 'const esc ' in line or 'let esc ' in line:
        print(f'esc defined at line {i+1}: {line.strip()[:120]}')

# Find where esc is first used
for i, line in enumerate(lines):
    if 'esc(' in line and 'function esc' not in line:
        print(f'esc first used at line {i+1}: {line.strip()[:120]}')
        break

# Find renderHomeTournaments call position
for i, line in enumerate(lines):
    if 'renderHomeTournaments()' in line and 'function' not in line:
        print(f'renderHomeTournaments() called at line {i+1}: {line.strip()[:120]}')
        
# Find populateSelect
for i, line in enumerate(lines):
    if 'function populateSelect' in line:
        print(f'populateSelect defined at line {i+1}: {line.strip()[:120]}')

# Now let's actually try running the JS with node to find runtime errors
# We need a minimal DOM mock
mock = """
const document = {
  getElementById: (id) => ({
    textContent: '', innerHTML: '', value: '', style: {display:''},
    classList: {add:()=>{},remove:()=>{},toggle:()=>{},contains:()=>false},
    addEventListener: ()=>{}, querySelector: ()=>null, querySelectorAll: ()=>({forEach:()=>{}}),
    dataset: {}, disabled: false, className: '',
  }),
  querySelectorAll: (s) => ({forEach: ()=>{}}),
  querySelector: (s) => document.getElementById('x'),
  addEventListener: ()=>{},
  body: {classList:{add:()=>{},remove:()=>{},toggle:()=>{}}},
  createElement: (t) => ({style:{},setAttribute:()=>{},appendChild:()=>{},
    classList:{add:()=>{},remove:()=>{},toggle:()=>{}}}),
};
const window = { location: {reload:()=>{}}, addEventListener:()=>{} };
const fetch = () => Promise.resolve({json:()=>Promise.resolve({ok:false})});
const alert = ()=>{};
const confirm = ()=>false;
const setTimeout = ()=>{};
const setInterval = ()=>0;
const clearInterval = ()=>{};
""" + js

with open('/tmp/check_runtime.js', 'w') as f:
    f.write(mock)

r2 = subprocess.run(['node', '/tmp/check_runtime.js'], capture_output=True, text=True, timeout=10)
print(f'\nNode runtime exit code: {r2.returncode}')
if r2.stderr:
    print(f'Runtime errors:\n{r2.stderr[:3000]}')
if r2.stdout:
    print(f'Output:\n{r2.stdout[:500]}')

esc defined at line 94: function esc(s) { const d = document.createElement('div'); d.textContent = s; return d.innerHTML; }
esc first used at line 140: <title>${esc(a.name)}: ${a.count} (${(pct*100).toFixed(1)}%)</title>
renderHomeTournaments() called at line 288: renderHomeTournaments();
renderHomeTournaments() called at line 295: renderHomeTournaments();
populateSelect defined at line 416: function populateSelect() {

Node runtime exit code: 0


In [32]:
import subprocess

# Regenerate (should have no warnings now)
r = subprocess.run(['/usr/bin/python3', 'generate_site.py'], capture_output=True, text=True, cwd='/workspaces/scrapermeleegg')
print('STDOUT:', r.stdout)
if r.stderr:
    print('STDERR:', r.stderr)
else:
    print('No warnings!')

# Commit and push
r2 = subprocess.run(['git', 'add', '-A'], capture_output=True, text=True, cwd='/workspaces/scrapermeleegg')
r3 = subprocess.run(['git', 'commit', '-m', 'feat: add per-pilot stats table in deck detail panel\n\n- Build per-player W-L-D stats from match data in build_metagame()\n- Group pilots by archetype, sorted by winrate\n- Display pilots table in Deck Detail panel with colored W/L/D/WR\n- Fix \\d regex escape warning in JS f-string'], 
    capture_output=True, text=True, cwd='/workspaces/scrapermeleegg')
print('\nCommit:', r3.stdout)
if r3.stderr:
    print('Commit stderr:', r3.stderr)

r4 = subprocess.run(['git', 'push'], capture_output=True, text=True, cwd='/workspaces/scrapermeleegg')
print('Push:', r4.stdout or r4.stderr)


STDOUT: [*] Cargando base de datos: /workspaces/scrapermeleegg/tournaments_db.json
[*] Procesando: Regional Championship - SCG CON Milwaukee - Season 4 - Round 2 - Saturday - 9:00 am
    Arquetipos con partidas: 104

[✓] Sitio generado en: dist/
    index.html  (830,936 bytes)
    Torneos embebidos: 1
      • Regional Championship - SCG CON Milwaukee - Season 4 - Round 2 - Saturday - 9:00 am (5046 matches)

No warnings!

Commit: [main e67e3d5] feat: add per-pilot stats table in deck detail panel
 2 files changed, 1384 insertions(+), 13 deletions(-)

Push: To https://github.com/torinescudo/scrapermeleegg
   a104103..e67e3d5  main -> main

